# Imports:

In [ ]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *


MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# files in use:

microbiome data:
MPA files from the folder 'Z:\Lab\Personal\ShaniBAF\MicrobiomeDataTables\FilteredAndMergedOnBD'
*PNP434 samples, not filtered on swabs/Genotek and on read counts. merged on BD using mean,
and sample outlier removal. no removal of same person!! total 423 samples
*PNP434 samples, Only swab samples, only samples with >9000000 reads postHGF. merged on BD using noOutlierMean,
and sample outlier removal. no removal of same person!! total 276 samples

in both binary and RA analysis, the input file is RA file, and then it is converted to binary within the function

TCR data:

binary analysis - sharingMatrix_moreThan50_434Samples
RA analysis -sharingMatrix_moreThan50_434Samples_RA 





### load and edit Mb file:


In [ ]:
#load BD_FD converter, to make sure we remove BD_FD columns from the MB file:
f1='%s/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolderAllSeqProjects_17042018'
BD_FD=pd.read_pickle(f1)

In [ ]:
# not filtered file:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_notfiltered_meannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'

Mb_PNP434_notFiltered=pd.read_pickle(f1)
Mb_PNP434_notFiltered=Mb_PNP434_notFiltered.set_index('BD') #### very important!!

In [ ]:
# filtered file:
f2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_Metabolon2_PNP434_filGenotek_fil9000000reads_meannSTD(3,)nMinSamples3_filterOutlierSampleFalse_filterSamePersonTrue'

Mb_PNP434_onlySwabs_only9Mreads=pd.read_pickle(f2)
Mb_PNP434_onlySwabs_only9Mreads=Mb_PNP434_onlySwabs_only9Mreads.set_index('BD') #### very important!!

In [ ]:
Mb_PNP434_notFiltered.head()

In [ ]:
Mb_PNP434_onlySwabs_only9Mreads.head()

In [ ]:
Mb_PNP434_notFiltered.columns.values[:5]

#### remove all columns that belong to BD_FD file:


In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolderMetabolon2_18032018'
BD_FD=pd.read_pickle(f1)

In [ ]:
#remove from non filtered file:
for column in BD_FD.columns.values:
    if column in Mb_PNP434_notFiltered.columns.values:
        print 'dropping %s column' %column
        Mb_PNP434_notFiltered=Mb_PNP434_notFiltered.drop(column,axis=1)

In [ ]:
#remove from filtered file:
for column in BD_FD.columns.values:
    if column in Mb_PNP434_onlySwabs_only9Mreads.columns.values:
        print 'dropping %s column' %column
        Mb_PNP434_onlySwabs_only9Mreads=Mb_PNP434_onlySwabs_only9Mreads.drop(column,axis=1)

### load and edit TCR file:

In [ ]:
# #load binary file::
file1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan50_434Samples'
sharingMatrix_moreThan50_434Samples = pd.read_pickle(file1)

In [ ]:
sharingMatrix_moreThan50_434Samples.head()

In [ ]:
#load RA file:
file1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan50_434Samples_RA'
sharingMatrix_moreThan50_434Samples_RA = pd.read_pickle(file1)

In [ ]:
sharingMatrix_moreThan50_434Samples_RA.head()

# binary TCR x binary mb species:

## run test for non-filtered Mb file -high thresholds just to test:

In [ ]:
MbDF=Mb_PNP434_notFiltered
MbDFName='Mb_PNP434_notFiltered'
TCRdf=sharingMatrix_moreThan50_434Samples
TCRdfName='TCRdf_pnp434_more50'
NsharedSamplesForSeqs=200
topNspecies=50

FisherResults_full=find_associations_binaryTCR_binaryMBspecies(MbDF, MbDFName,TCRdf, TCRdfName,NsharedSamplesForSeqs, topNspecies)

## run test for non-filtered Mb file:

In [ ]:
MbDF=Mb_PNP434_notFiltered
MbDFName='Mb_PNP434_metabolon2_notFiltered'
TCRdf=sharingMatrix_moreThan50_434Samples
TCRdfName='TCRdf_pnp434_more50'
NsharedSamplesForSeqs=50
topNspecies=100

FisherResults_full=find_associations_binaryTCR_binaryMBspecies(MbDF, MbDFName,TCRdf, TCRdfName,NsharedSamplesForSeqs, topNspecies)

## run test for swab and minimal reads filtered file:

In [ ]:
MbDF=Mb_PNP434_onlySwabs_only9Mreads
MbDFName='Mb_PNP434_metabolon2_onlySwabs_only9Mreads'
TCRdf=sharingMatrix_moreThan50_434Samples
TCRdfName='TCRdf_pnp434_more50'
NsharedSamplesForSeqs=50
topNspecies=100

FisherResults_full=find_associations_binaryTCR_binaryMBspecies(MbDF, MbDFName,TCRdf, TCRdfName, NsharedSamplesForSeqs, topNspecies)

## explore result file:

### filtered mb file:

In [ ]:
#check number of tests with filtered file:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/countTests_fisher_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_50_100','rb') as fp:
    countTestFiltered=pickle.load(fp)
countTestFiltered    

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/FisherResults_full_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_seqsSharedMoreThan50_top100Species.xlsx'
FisherResults_full_filtered=pd.read_excel(file1)

In [ ]:
FisherResults_full_filtered.head()

In [ ]:
print float(len(FisherResults_full_filtered))/countTestFiltered  

In [ ]:
# look for species with the highest number of associated TCR sequences:
FisherResults_full_filtered.groupby('species').seq.nunique().sort_values(ascending=False).head(10)

In [ ]:
# look for TCR sequences with the highest number of associated species :
FisherResults_full_filtered.groupby('seq').species.nunique().sort_values(ascending=False).head(10)

#### add ratio columns:

In [ ]:
FisherResults_full_filtered['sp0seq1/sp1seq1']=FisherResults_full_filtered['speciesNegSeqPos']/FisherResults_full_filtered['speciesPosSeqPos']

In [ ]:
FisherResults_full_filtered['sp0seq0/sp1seq0']=FisherResults_full_filtered['speciesNegSeqNeg']/FisherResults_full_filtered['speciesPosSeqNeg']

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/FisherResults_full.xlsx'
FisherResults_full_filtered.to_excel(file1)

#### 'type 1 associations' - sequence prevents species:
sp0seq1 is high, sp1seq0 might be high, sp1seq0 is indifferent, sp1seq1 should be very low

OR<1

In [ ]:

FisherResults_full_filtered[FisherResults_full_filtered['speciesPosSeqPos']<8]

In [ ]:

FisherResults_full_filtered[(FisherResults_full_filtered['speciesPosSeqPos']<10)&(FisherResults_full_filtered['OR']<0.3)]

In [ ]:
FisherResults_full_filtered[FisherResults_full_filtered['OR']!=0].sort_values(by='OR').head(10)

In [ ]:
FisherResults_full_filtered[(FisherResults_full_filtered['OR']>0)&(FisherResults_full_filtered['OR']<1)].sort_values(by='p').head(10)

In [ ]:
FisherResults_full_filtered[FisherResults_full_filtered['OR']<1000].sort_values(by='OR',ascending=False).head(10)

In [ ]:
FisherResults_full_filtered[FisherResults_full_filtered['OR']<1000].sort_values(by='p').head(10)

In [ ]:
FisherResults_full_filtered.sort_values(by='OR').tail(10)

#### 'type 2 associations' - species presence leads to sequence presence:
no species, no sequence
sp0seq0 is high, sp1seq1 might be high, sp1seq0 and sp0seq1 are rare

OR>1



In [ ]:

FisherResults_full_filtered[FisherResults_full_filtered['speciesPosSeqNeg']<40]

In [ ]:
# look for associations in which there are no cases of sequences present when there is no species. sort them to show those cases
# in which there are more seq-neg,species neg cases in comparison to seq-neg, spe-pos cases:
FisherResults_full_filtered[(FisherResults_full_filtered['speciesNegSeqPos']==0)].sort_values(by='sp0seq0/sp1seq0', ascending=False).head(10)

In [ ]:
print len(FisherResults_full_filtered[(FisherResults_full_filtered['speciesNegSeqPos']==0)])

In [ ]:
#look for associations in which there are no cases of spe-pos,seq-neg:
#there are no such cases
FisherResults_full_filtered[(FisherResults_full_filtered['speciesPosSeqNeg']==0)]

In [ ]:
# what is the maximal OR?
pd.DataFrame(FisherResults_full_filtered['OR'].unique()).sort_values(by=0,ascending=False).head()

In [ ]:
#look for assocaition with highest OR which is not inf:
FisherResults_full_filtered[(FisherResults_full_filtered['OR']<14)&(FisherResults_full_filtered['OR']>10)]

#### calculate the ratio between the two types:


In [ ]:
type1_filtered= len(FisherResults_full_filtered[FisherResults_full_filtered['OR']<1])
type1_filtered

In [ ]:
type2_filtered= len(FisherResults_full_filtered[FisherResults_full_filtered['OR']>1])
type2_filtered

In [ ]:
ratio1_filtered=float(type1_filtered)*100/(type1_filtered+type2_filtered)
ratio1_filtered

In [ ]:
ratio2_filtered=float(type2_filtered)*100/(type1_filtered+type2_filtered)
ratio2_filtered

In [ ]:
FisherResults_full_filtered[FisherResults_full_filtered['OR']<1]['p'].mean()

In [ ]:
FisherResults_full_filtered[FisherResults_full_filtered['OR']>1]['p'].mean()

### non filtered mb file:

In [ ]:
#check number of tests with filtered file:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/countTests_fisher_Mb_PNP434_metabolon2_notFiltered_TCRdf_pnp434_more50_50_100','rb') as fp:
    countTestNotFiltered=pickle.load(fp)
countTestNotFiltered    

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/FisherResults_full_Mb_PNP434_metabolon2_notFiltered_TCRdf_pnp434_more50_seqsSharedMoreThan50_top100Species.xlsx'
FisherResults_full_notFiltered=pd.read_excel(file1)

In [ ]:
FisherResults_full_notFiltered.head()

In [ ]:
print float(len(FisherResults_full_notFiltered))/countTestNotFiltered   

In [ ]:
# look for species with the highest number of associated TCR sequences:
FisherResults_full_notFiltered.groupby('species').seq.nunique().sort_values(ascending=False).head(10)

In [ ]:
# look for TCR sequences with the highest number of associated species :
FisherResults_full_notFiltered.groupby('seq').species.nunique().sort_values(ascending=False).head(10)

#### add ratio columns:

In [ ]:
FisherResults_full_notFiltered['sp0seq1/sp1seq1']=FisherResults_full_notFiltered['speciesNegSeqPos']/FisherResults_full_notFiltered['speciesPosSeqPos']

In [ ]:
FisherResults_full_notFiltered['sp0seq0/sp1seq0']=FisherResults_full_notFiltered['speciesNegSeqNeg']/FisherResults_full_notFiltered['speciesPosSeqNeg']

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/FisherResults_full_notFiltered.xlsx'
FisherResults_full_notFiltered.to_excel(file1)

#### 'type 1 associations' - sequence prevents species:
sp0seq1 is high, sp1seq0 might be high, sp1seq0 is indifferent, sp1seq1 should be very low

OR<1

In [ ]:

FisherResults_full_notFiltered[FisherResults_full_notFiltered['speciesPosSeqPos']<8]

In [ ]:

FisherResults_full_notFiltered[(FisherResults_full_notFiltered['speciesPosSeqPos']<10)&(FisherResults_full_notFiltered['OR']<0.3)]

In [ ]:
FisherResults_full_notFiltered.sort_values(by='OR').head(10)

In [ ]:
FisherResults_full_notFiltered.sort_values(by='OR').tail()

In [ ]:
FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']<1].sort_values(by='p').head(10)

#### 'type 2 associations' - species presence leads to sequence presence:
no species, no sequence
sp0seq0 is high, sp1seq1 might be high, sp1seq0 and sp0seq1 are rare

OR>1



In [ ]:

FisherResults_full_notFiltered[FisherResults_full_notFiltered['speciesPosSeqNeg']<40]

In [ ]:
# look for associations in which there are no cases of sequences present when there is no species. sort them to show those cases
# in which there are more seq-neg,species neg cases in comparison to seq-neg, spe-pos cases:
FisherResults_full_notFiltered[(FisherResults_full_notFiltered['speciesNegSeqPos']==0)].sort_values(by='sp0seq0/sp1seq0', ascending=False).head(10)

In [ ]:
print len(FisherResults_full_notFiltered[(FisherResults_full_notFiltered['speciesNegSeqPos']==0)])

In [ ]:
#look for associations in which there are no cases of spe-pos,seq-neg:
#there are no such cases
FisherResults_full_notFiltered[(FisherResults_full_notFiltered['speciesPosSeqNeg']==0)]

In [ ]:
# what is the maximal OR?
pd.DataFrame(FisherResults_full_notFiltered['OR'].unique()).sort_values(by=0,ascending=False).head()

In [ ]:
#look for assocaition with highest OR which is not inf:
FisherResults_full_notFiltered[(FisherResults_full_notFiltered['OR']<14)&(FisherResults_full_notFiltered['OR']>10)]

In [ ]:
FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']>1].sort_values(by='p').head(10)

#### calculate the ratio between the two types:


In [ ]:
type1_notFiltered= len(FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']<1])
type1_notFiltered

In [ ]:
type2_notFiltered= len(FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']>1])
type2_notFiltered

In [ ]:
ratio1_notFiltered=float(type1_notFiltered)*100/(type1_notFiltered+type2_notFiltered)
ratio1_notFiltered

In [ ]:
ratio2_notFiltered=float(type2_notFiltered)*100/(type1_notFiltered+type2_notFiltered)
ratio2_notFiltered

In [ ]:
FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']<1]['p'].mean()

In [ ]:
FisherResults_full_notFiltered[FisherResults_full_notFiltered['OR']>1]['p'].mean()

# RA TCR x RA mb species

## run test for non-filtered Mb file:

In [ ]:
###use the TCRdf_RA file this time!

MbDF=Mb_PNP434_notFiltered
MbDFName='Mb_PNP434_metabolon2_notFiltered'
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
TCRdfName='TCRdf_pnp434_more50_RA'
NsharedSamplesForSeqs=50
topNspecies=100

find_associations_TCR_RA_MBspecies_RA(MbDF, MbDFName,TCRdf_RA, TCRdfName,NsharedSamplesForSeqs=50,topNspecies=100,outlierSTD=3)


## run test for for swab and minimal reads filtered file:

In [ ]:
###use the TCRdf_RA file this time!

MbDF=Mb_PNP434_onlySwabs_only9Mreads
MbDFName='Mb_PNP434_metabolon2_onlySwabs_only9Mreads'
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
TCRdfName='TCRdf_pnp434_more50_RA'
NsharedSamplesForSeqs=50
topNspecies=100

find_associations_TCR_RA_MBspecies_RA(MbDF, MbDFName,TCRdf_RA, TCRdfName,NsharedSamplesForSeqs=50,topNspecies=100,outlierSTD=3)


## explore results:

## filtered Mb:

In [ ]:

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/PearsonResults_full_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_RA_seqsSharedMoreThan50_top100Species_outlierSTD3.xlsx'
Pearson_filtered=pd.read_excel(file1)

In [ ]:
#check number of tests with filtered file:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/countTests_pearson_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_RA_50_100_3' , 'rb') as fp:
    countTestPearsonFiltered=pickle.load(fp)
countTestPearsonFiltered    

In [ ]:
df=Pearson_filtered
df.head()

In [ ]:
print len(df[df['Sig by bonferroni corrected pVal']==1])

In [ ]:
print len(df[df['sig. by FDR=0.00022']==1])

In [ ]:
print len(df[df['sig. by FDR=0.01']==1])

In [ ]:
# look for species with the highest number of associated TCR sequences:
df.groupby('species').seq.nunique().sort_values(ascending=False).head(10)

In [ ]:
# look for TCR sequences with the highest number of associated species :
df.groupby('seq').species.nunique().sort_values(ascending=False).head(10)

### type 1 and type 2 associations:

In [ ]:
type1= len(df[df['r']<0])
type1

In [ ]:
type2= len(df[df['r']>0])
type2

In [ ]:
ratio1=float(type1)*100/(type1+type2)
ratio1

In [ ]:
ratio2=float(type2)*100/(type1+type2)
ratio2

In [ ]:
df[df['r']<0]['p'].mean()

In [ ]:
df[df['r']>0]['p'].mean()

### most significant 'p's and 'r's:

In [ ]:
df[df['r']>0].sort_values(by='p').head(10)

In [ ]:
df[df['r']<0].sort_values(by='p').head(10)

In [ ]:
df.sort_values(by='r').head(10)

In [ ]:
df.sort_values(by='r',ascending=False).head(10)

### check manually some results:


In [ ]:
seq='CASSVGQGAYEQYF'
species='Lachnospiraceae_bacterium_5_1_63FAA'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSLQVNTEAFF'
species='Clostridium_bolteae'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CSAREREYEQYF'
species='Clostridium_bartlettii'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSLTSGSTDTQYF'
species='Sutterella_wadsworthensis'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSLRGNEQFF'
species='Streptococcus_salivarius'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPSGANVLTF'
species='Bifidobacterium_longum'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPSGANVLTF'
species='Bifidobacterium_longum'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSLGGGYNEQFF'
species='Parabacteroides_merdae'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPGGNYGYTF'
species='Collinsella_aerofaciens'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSFGGTEAFF'
species='Roseburia_inulinivorans'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSRGNTEAFF'
species='Dorea_formicigenerans'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSSGDTQYF'
species='Dorea_formicigenerans'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSQSYEQYF'
species='Eubacterium_hallii'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSLGQGYGYTF'
species='Alistipes_shahii'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPYQETQYF'
species='Erysipelotrichaceae_bacterium_6_1_45'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPYQETQYF'
species='Bacteroides_intestinalis'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


# merge result files:

In [ ]:
#load pearson results:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/PearsonResults_full_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_RA_seqsSharedMoreThan50_top100Species_outlierSTD3.xlsx'
Pearson_filtered=pd.read_excel(file1)
print len(Pearson_filtered)
Pearson_filtered.head()

In [ ]:
#load fisher results:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/FisherResults_full_Mb_PNP434_metabolon2_onlySwabs_only9Mreads_TCRdf_pnp434_more50_seqsSharedMoreThan50_top100Species.xlsx'
FisherResults_full_filtered=pd.read_excel(file1)
print len(FisherResults_full_filtered)
FisherResults_full_filtered.head()

In [ ]:
#merge:
merged_fisher_pearson=pd.merge(Pearson_filtered,
                               FisherResults_full_filtered,how='inner',
                              left_on=('species','seq'),right_on=('species','seq'))
print len(merged_fisher_pearson)
for column in merged_fisher_pearson:
    newColumn=column.replace('_x','_pearson')
    newColumn=newColumn.replace('_y','_Fisher')
    merged_fisher_pearson=merged_fisher_pearson.rename(columns={column:newColumn})
merged_fisher_pearson=merged_fisher_pearson.sort_values(by='p_pearson')
merged_fisher_pearson.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/merged_fisher_pearson_seqMoreThan50_top100speciesFiltered_.xlsx'
merged_fisher_pearson.to_excel(file1)

## generate a figure with result examples:
2 for negative correlations (sig. in both pearson and Fisher)
2 for positive correlations (sig. in both pearson and Fisher)
two for non-significant correlations

In [ ]:
color_binary=True
posSeqList=['CASSLTSGSTDTQYF','CASSLEDTQYF']
posSpeciesList=['Sutterella_wadsworthensis','Bacteroides_salyersiae']
negSeqList=['CASSPGGNYGYTF','CASSITDTQYF']
negSpeciesList=['Collinsella_aerofaciens','Barnesiella_intestinihominis']
uncorrelatedSeqList=['CASSSDSYEQYF','CASSSDSYEQYF']
uncorrelatedSpeciesList=['Alistipes_finegoldii','Parabacteroides_distasonis']

MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA

fig, axes=plt.subplots(nrows=2, ncols=3,figsize=(20,10))

ax=axes[0,0]
seq=posSeqList[0]
species=posSpeciesList[0]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.set_title('Positive correlation', fontsize=20)
                         
ax=axes[1,0]
seq=posSeqList[1]
species=posSpeciesList[1]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)

ax=axes[0,1]
seq=negSeqList[0]
species=negSpeciesList[0]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.set_title('Negative correlation', fontsize=20)

ax=axes[1,1]
seq=negSeqList[1]
species=negSpeciesList[1]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
                         
ax=axes[0,2]
seq=uncorrelatedSeqList[0]
species=uncorrelatedSpeciesList[0]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.set_title('No correlation', fontsize=20)

ax=axes[1,2]
seq=uncorrelatedSeqList[1]
species=uncorrelatedSpeciesList[1]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)

plt.subplots_adjust(left=0.02, bottom=None, right=0.99, top=0.84,
                wspace=0.2, hspace=0.25)
plt.show()


In [ ]:
color_binary=True
posSeqList=['CASSLTSGSTDTQYF','CASSLEDTQYF']
posSpeciesList=['Sutterella_wadsworthensis','Bacteroides_salyersiae']
negSeqList=['CASSPGGNYGYTF','CASSITDTQYF']
negSpeciesList=['Collinsella_aerofaciens','Barnesiella_intestinihominis']


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA

fig, axes=plt.subplots(nrows=2, ncols=2,figsize=(14,10))

ax=axes[0,0]
seq=posSeqList[0]
species=posSpeciesList[0]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.set_title('Positive correlation', fontsize=26)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)
                         
ax=axes[1,0]
seq=posSeqList[1]
species=posSpeciesList[1]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)

ax=axes[0,1]
seq=negSeqList[0]
species=negSpeciesList[0]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.set_title('Negative correlation', fontsize=26)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)

ax=axes[1,1]
seq=negSeqList[1]
species=negSpeciesList[1]
plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)
                         

plt.subplots_adjust(left=0.02, bottom=None, right=0.99, top=0.8,
                wspace=0.22, hspace=0.3)
plt.show()


## add identity information to the table:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/merged_fisher_pearson_seqMoreThan50_top100speciesFiltered_.xlsx'
merged_fisher_pearson=pd.read_excel(file1)
merged_fisher_pearson.head()

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_22022018'
CDR3identityTable=pd.read_pickle(file1)
CDR3identityTable.head()

In [ ]:
merged_fisher_pearson_withIdentity=pd.merge(merged_fisher_pearson,CDR3identityTable,how='left',
                                           left_on='seq',right_index=True)
print len(merged_fisher_pearson)
print len(CDR3identityTable)
print len(merged_fisher_pearson_withIdentity)
merged_fisher_pearson_withIdentity.head()

In [ ]:
merged_fisher_pearson_withIdentity[merged_fisher_pearson_withIdentity['Epitope species_VDJDB'].notnull()].head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/merged_fisher_pearson_withIdentity_filteredMb'
merged_fisher_pearson_withIdentity.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/merged_fisher_pearson_withIdentity_filteredMb.xlsx'
merged_fisher_pearson_withIdentity.to_excel(file2)



### figures for pairs identified by the merged analysis and TCR identity:

In [ ]:
seq='CASSRLAGGTDTQYF'
species='Clostridium_leptum'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSRLAGGTDTQYF'
species='Clostridium_leptum'
MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
fig,ax=plt.subplots()

plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)
plt.show()

In [ ]:
seq='CASSPYQETQYF'
species='Streptococcus_salivarius'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)
ax.yaxis.label.set_size(18)
ax.xaxis.label.set_size(18)


In [ ]:
seq='CASSPYQETQYF'
species='Streptococcus_salivarius'
MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
fig,ax=plt.subplots()

plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(22)
ax.xaxis.label.set_size(22)
plt.show()

In [ ]:
seq='CASSPGTGELFF'
species='Veillonella_unclassified'


MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdfRA=sharingMatrix_moreThan50_434Samples_RA

check_seq_species_relations_sns(TCRdfRA,MbDF,seq, species, color_binary=True)


In [ ]:
seq='CASSPGTGELFF'
species='Veillonella_unclassified'
MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
fig,ax=plt.subplots()

plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(22)
ax.xaxis.label.set_size(22)
plt.show()

In [ ]:
seq='CASSETSGGTDTQYF'
species='Alistipes_senegalensis'
MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
fig,ax=plt.subplots()

plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(22)
ax.xaxis.label.set_size(22)
plt.show()

In [ ]:
seq='CASSLGGGSYEQYF'
species='Alistipes_senegalensis'
MbDF=Mb_PNP434_onlySwabs_only9Mreads
TCRdf_RA=sharingMatrix_moreThan50_434Samples_RA
fig,ax=plt.subplots()

plot_seq_species_scatter(MbDF,TCRdf_RA,ax, seq, species, color_binary)
ax.yaxis.label.set_size(22)
ax.xaxis.label.set_size(22)
plt.show()

# try to updated pearson function 
from TCR_microbiome_interactions_functions.py 22.3.18

In [ ]:
MbDF_RA=Mb_PNP434_notFiltered
MbDFName='Mb_PNP434_notFiltered'
TCRdf_RA=sharingMatrix_moreThan50_434Samples
TCRdfName='TCRdf_pnp434_more50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=10
topNseqs=10
outlierSTD=None
completePearson=True
pearsonForHeatMap=False
resultFolder='%s/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/CorrelResults_%s_%s_NsharedSpecies%s_Nsharedseqs%s_topSpecies%s\
_topSeqs%s_outlierSTD%s' %(MyPath,MbDFName,TCRdfName,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD)


find_associations_TCR_RA_MBspecies_RA(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD,resultFolder,completePearson,pearsonForHeatMap)



In [ ]:
MbDF_RA=Mb_PNP434_notFiltered
MbDFName='MbPNP434notFiltered'
TCRdf_RA=sharingMatrix_moreThan50_434Samples
TCRdfName='TCRdfPNP434more50'
NsharedSamplesForSpecies=200
NsharedSamplesForSeqs=200
topNspecies=None
topNseqs=None
outlierSTD=None
completePearson=False
pearsonForHeatMap=True
resultFolder='%s/TCR_real_data/PhenotypicData/Microbiome-TCR interactions/CorrelResults_%s_%s_NsharedSpecies%s_Nsharedseqs%s_topSpecies%s\
_topSeqs%s_outlierSTD%s' %(MyPath,MbDFName,TCRdfName,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD)


find_associations_TCR_RA_MBspecies_RA(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD,resultFolder,completePearson,pearsonForHeatMap)

